### Generates Vectos from unique names from source and creates search indexes

In [1]:
import numpy as np 
import pandas as pd 
import pyphi
import requests
import os
import falconn
import random

In [2]:
def query_falconn_index(name, lhs_table, number, names, vectors):
    query = call_embedding_ws(name)
    response = lhs_table.find_k_nearest_neighbors(query[0], k=number)
    
    return process_lhs_table_response(query, response, names, vectors)

def process_lhs_table_response(query, response, names, vectors):
    df = pd.DataFrame(index=range(len(response)), columns=['id', 'name', 'cosine'])   
    i = 0
    
    for resp in response:
        name = names.get_value(resp, 'name')
        cos = cos_similarity(query, vectors[resp])
        df.set_value(index=i, col='id', value=resp)
        df.set_value(index=i, col='name', value=name)
        df.set_value(index=i, col='cosine', value=cos)
        i = i + 1
     
    df = df.sort_values(by='cosine', ascending=False)
    df = df.reset_index(drop=True)

    return df

def cos_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)
    return dot_product

def call_embedding_ws(names):
    names = [item.lower() for item in names]
    
#    url = 'http://54.36.53.127:8009/embedding'
    url = 'http://127.0.0.1:8009/embedding'
    headers = {"content-type": "application/json"}
    response = requests.post(url, json=names)
    
    arr = pyphi.jsonify.loads(response.text)
    x = np.array(arr)
    x = x.astype(dtype=np.float32)
    
    return x

input_path = 'data/src_distinct_names/src'
output_path = 'data/src_distinct_names'

#### Calculate vectors for names in source files to output files

In [ ]:
%%time
directory = os.fsencode(input_path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        names = pd.read_csv(input_path + '/' + filename, sep='|', names=['name'], header=None)
#        np.where(pd.isnull(names))
#        names.iloc[245193]
        
        i = -1
        j = -1
        ws_params = []

        file = open(output_path + '/vectors_' + filename,"w") 

        for index, row in names.iterrows():
            ws_params.append(row['name'].strip().lower())
            i = i + 1
            if i % 512 == 0:
                ws_result = call_embedding_ws(ws_params)
                for vect in ws_result:
                    j = j + 1
                   
                    v_str = np.array2string(vect, separator=',', max_line_width=10000).strip("[]").replace(" ", "")
                    
                    #print("{n}|{v}\n".format(n=names['name'][j], v=vect[:5]))
                    file.write("{n}|{v}\n".format(n=names['name'][j], v=v_str))
                ws_params = []
                print(i)

        if len(ws_params) > 0:
            ws_result = call_embedding_ws(ws_params)
            for vect in ws_result:
                j = j + 1
                v_str = np.array2string(vect, separator=',', max_line_width=10000).strip("[]").replace(" ", "")
                file.write("{n}|{v}\n".format(n=names['name'][j], v=v_str))

        file.close()

In [3]:
%%time
directory = os.fsencode(input_path)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        names = pd.read_csv(input_path + '/' + filename, sep='|', names=['name'], header=None)
#        np.where(pd.isnull(names))
#        names.iloc[245193]
        
        ws_params = []
        file = open(output_path + '/vectors_' + filename,"w") 

        for index, row in names.iterrows():
            ws_params.append(row['name'].strip().lower())
            ws_result = call_embedding_ws(ws_params)
            v_str = np.array2string(ws_result[0], separator=',', max_line_width=10000).strip("[]").replace(" ", "")
            file.write("{n}|{v}\n".format(n=row['name'], v=v_str))
            ws_params = []
        file.close()

family_names_master_arb_distinct.csv
family_names_master_eng_distinct.csv
given_names_dan_arb_distinct.csv
given_names_dan_eng_distinct.csv
given_names_master_arb_distinct.csv
given_names_master_eng_distinct.csv
CPU times: user 1h 11min 44s, sys: 1min 47s, total: 1h 13min 31s
Wall time: 2h 10min 27s


#### Read vector files and parse vector strings

In [4]:
directory = os.fsencode(output_path)
names_dict = {}
vectors_dict = {}
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        print(filename)
        
        names_dict[filename] = pd.read_csv(output_path + '/' + filename, sep='|', names=['name', 'vector_string'], header=None)
        
        vector_strings = names_dict[filename]['vector_string'].as_matrix()
        vectors = np.zeros(shape=(len(vector_strings),256))
        i = 0
        for v in vector_strings:
            try:
                x = np.fromstring(v, dtype=np.float32, sep=',')
                vectors[i] = x
                i = i+1
            except:
                print("{i} {v}".format(i=i, v=v))
                raise
            
        vectors_dict[filename] = vectors.astype(dtype=np.float32)

vectors_given_names_master_eng_distinct.csv
vectors_family_names_master_arb_distinct.csv
vectors_family_names_master_eng_distinct.csv
vectors_given_names_dan_arb_distinct.csv
vectors_given_names_dan_eng_distinct.csv
vectors_given_names_master_arb_distinct.csv


#### Merge english and arabic names separately

In [5]:
eng_names_merged =  pd.DataFrame(columns=['name', 'vector_string'])
arb_names_merged = pd.DataFrame(columns=['name', 'vector_string'])


for key, value in names_dict.items():
    if 'eng' in key:
        eng_names_merged = eng_names_merged.append(value).reset_index(drop=True)

eng_names_merged = eng_names_merged.drop_duplicates(['name']).reset_index(drop=True)
eng_names_merged = eng_names_merged.reset_index(drop=True)
        
vector_strings = eng_names_merged['vector_string'].as_matrix()
eng_vectors_merged = np.zeros(shape=(len(vector_strings),256))
i = 0
for v in vector_strings:
    x = np.fromstring(v, dtype=np.float32, sep=',')
    eng_vectors_merged[i] = x
    i = i+1
    
eng_vectors_merged = eng_vectors_merged.astype(dtype=np.float32)



for key, value in names_dict.items():
    if 'arb' in key:
        arb_names_merged = arb_names_merged.append(value).reset_index(drop=True)

arb_names_merged = arb_names_merged.drop_duplicates(['name']).reset_index(drop=True)
arb_names_merged = arb_names_merged.reset_index(drop=True)
        
vector_strings = arb_names_merged['vector_string'].as_matrix()
arb_vectors_merged = np.zeros(shape=(len(vector_strings),256))
i = 0
for v in vector_strings:
    x = np.fromstring(v, dtype=np.float32, sep=',')
    arb_vectors_merged[i] = x
    i = i+1
arb_vectors_merged = arb_vectors_merged.astype(dtype=np.float32) 


#### Validate vectors

In [12]:
for s in random.sample(range(0, len(arb_names_merged)), 2000):
    name = arb_names_merged.iloc[s]['name']
    control_vect = call_embedding_ws([name])[0]
    vect = np.fromstring(arb_names_merged.iloc[s]['vector_string'], dtype=np.float32, sep=',')
    test = control_vect != vect
    if test.any == False:
        print(name)
        print(control_vect == vect)
    
for s in random.sample(range(0, len(eng_names_merged)), 2000):
    name = eng_names_merged.iloc[s]['name']
    control_vect = call_embedding_ws([name])[0]
    vect = np.fromstring(eng_names_merged.iloc[s]['vector_string'], dtype=np.float32, sep=',')
    test = control_vect != vect
    if test.any == False:
        print(name)
        print(control_vect == vect)

#### Build Falconn indexes

In [13]:
%%time
number_of_tables = 50
assert eng_vectors_merged.dtype == np.float32
assert arb_vectors_merged.dtype == np.float32

params_cp = falconn.LSHConstructionParameters()
params_cp.dimension = len(eng_vectors_merged[0])
params_cp.lsh_family = falconn.LSHFamily.CrossPolytope
params_cp.distance_function = falconn.DistanceFunction.EuclideanSquared
params_cp.l = number_of_tables
# we set one rotation, since the data is dense enough,
# for sparse data set it to 2
params_cp.num_rotations = 2
params_cp.seed = 5721840
# we want to use all the available threads to set up
params_cp.num_setup_threads = 0
params_cp.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
# we build 24-bit hashes so that each table has
# 2^24 bins; this is a good choise since 2^24 is of the same
# order of magnitude as the number of data points
falconn.compute_number_of_hash_functions(18, params_cp)

eng_merged_table = falconn.LSHIndex(params_cp)
eng_merged_table.setup(eng_vectors_merged)

eng_merged_query_object = eng_merged_table.construct_query_object()




params_cp = falconn.LSHConstructionParameters()
params_cp.dimension = len(arb_vectors_merged[0])
params_cp.lsh_family = falconn.LSHFamily.CrossPolytope
params_cp.distance_function = falconn.DistanceFunction.EuclideanSquared
params_cp.l = number_of_tables
# we set one rotation, since the data is dense enough,
# for sparse data set it to 2
params_cp.num_rotations = 2
params_cp.seed = 5721840
# we want to use all the available threads to set up
params_cp.num_setup_threads = 0
params_cp.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
# we build 24-bit hashes so that each table has
# 2^24 bins; this is a good choise since 2^24 is of the same
# order of magnitude as the number of data points
falconn.compute_number_of_hash_functions(18, params_cp)

arb_merged_table = falconn.LSHIndex(params_cp)
arb_merged_table.setup(arb_vectors_merged)

arb_merged_query_object = arb_merged_table.construct_query_object()

CPU times: user 53.3 s, sys: 119 ms, total: 53.4 s
Wall time: 13.9 s


In [21]:
query_falconn_index(['jones'], eng_merged_query_object, 20, eng_names_merged, eng_vectors_merged)

,id,name,cosine
0,147053,JONES,[1.0]
1,115278,GUNES,[0.999886]
2,147250,JONIK,[0.999876]
3,151183,JUNKI,[0.999847]
4,151326,JUNWEI,[0.99973]
5,141973,JEANCY,[0.999656]
6,145008,JINCY,[0.999604]
7,145249,JINKI,[0.999551]
8,141166,JAUNKA,[0.999549]
9,145259,JINKY,[0.99953]


In [15]:
query_falconn_index(['hamza'], arb_merged_query_object, 20, arb_names_merged, arb_vectors_merged)

,id,name,cosine
0,95803,جمكو,[0.999957]
1,32591,أهمات,[0.999947]
2,182657,شموطى,[0.999924]
3,162879,سميده,[0.999923]
4,95859,جموعى,[0.999922]
5,162880,سميدى,[0.999921]
6,162774,سموده,[0.999918]
7,16251,اكميت,[0.999913]
8,162842,سميثى,[0.999908]
9,162585,سمتي,[0.999908]
